# 14_NestJS_Built_in_Pipes

This note explains **how NestJS built-in pipes transform incoming request data** *before* it reaches the controller, focusing mainly on:

* `ParseIntPipe`
* `DefaultValuePipe`

Once you understand these two, the other built-in pipes become very easy to use.

---

## 1. What are Built-in Pipes in NestJS?

* **Pipes** in NestJS are used to:

  * **Transform** incoming data (e.g., string → number)
  * **Validate** incoming data
* Pipes run **before** the request reaches the controller method.
* NestJS provides **9 built-in pipes** in total.
* Among them, there is a special one: **`ValidationPipe`**, usually used together with **DTOs** and **custom pipes**.

  * In this lesson, `ValidationPipe` is *mentioned* but not used. It will be used later when creating custom pipes.
* The other **8 built-in pipes** can be used directly on:

  * **Route parameters**
  * **Query strings**

Some commonly used built-in pipes:

* `ParseIntPipe` → converts value to **integer**
* `ParseFloatPipe` → converts value to **float**
* `ParseBoolPipe` → converts value to **boolean**
* `ParseArrayPipe` → converts value to **array**
* `DefaultValuePipe` → sets a **default value** if none is provided

> In this note, we focus on: **`ParseIntPipe`** and **`DefaultValuePipe`**.

---

## 2. Using Built-in Pipes – Importing

Before using any built-in pipe, you must **import** it from:

```ts
import {
  Controller,
  Get,
  Param,
  ParseIntPipe,
  Post,
  Query,
  DefaultValuePipe
} from '@nestjs/common';
```

All these decorators and pipes come from **`@nestjs/common`**.

---

## 3. Example Controller Setup

```ts
@Controller('users')
export class UsersController {

  @Get()
  getUsers(
    @Query('limit', new DefaultValuePipe(10), ParseIntPipe) limit: number,
    @Query('page', new DefaultValuePipe(1), ParseIntPipe) page: number,
  ) {
    const usersService = new UsersService();
    console.log(limit, page);
    return usersService.getAllUsers();
  }

  @Get(':id')
  getUserById(@Param('id', ParseIntPipe) id: number) {
    const userService = new UsersService();
    console.log(typeof id, id);
    return userService.getUserById(id);
  }

  @Post()
  createUser() {
    const user = { id: 3, name: 'Merry', age: 25, gender: 'female', isMarried: false };
    const userService = new UsersService();
    userService.createUser(user);
    return 'User created successfully';
  }
}
```

We’ll break down what’s happening here step by step.

---

## 4. `ParseIntPipe` with Route Parameters (`@Param`)

### 4.1 Problem Without Pipes

* Consider the route:

  ```ts
  @Get(':id')
  getUserById(@Param('id') id: string) {
    // id is a string here
  }
  ```

* When you call:

  * `GET /users/2`

* The value of `id` will be **read as a string** → `'2'`.

* But in your service, you may expect a **number**:

  ```ts
  getUserById(id: number) { /* ... */ }
  ```

* Without pipes, you would manually convert:

  ```ts
  const numericId = +id; // or Number(id)
  ```

### 4.2 Using `ParseIntPipe`

Instead of converting manually, you can let NestJS handle the conversion **before** the controller method runs.

```ts
@Get(':id')
getUserById(@Param('id', ParseIntPipe) id: number) {
  const userService = new UsersService();
  console.log(typeof id, id);
  return userService.getUserById(id);
}
```

#### What happens here?

* `@Param('id', ParseIntPipe)`:

  * Reads the `id` route parameter from the URL.
  * Applies `ParseIntPipe` to it.
  * `ParseIntPipe` tries to **convert the value to an integer**.
* If conversion succeeds:

  * The converted **number** is assigned to the `id` parameter.
  * `typeof id` will be `'number'`.
* If conversion fails (e.g., `GET /users/abc`):

  * `ParseIntPipe` **throws an exception**.
  * Response:

    * **HTTP status code:** `400 Bad Request`
    * Error message: `"Bad Request"` (because the value cannot be converted to a number).

### 4.3 Behaviour Example

* Request: `GET /users/abc`

  * `abc` cannot be converted to a number → `ParseIntPipe` throws error.
  * Response: `400 Bad Request`.

* Request: `GET /users/100`

  * `100` → successfully parsed as `number`.
  * If no user with ID `100`, you might get empty result or `undefined`, but **no pipe error**.

> ✅ Conclusion: `ParseIntPipe` helps ensure that route parameters like `:id` are always **numbers** before your logic runs.

---

## 5. `ParseIntPipe` with Query Parameters (`@Query`)

Now we use `ParseIntPipe` with **query strings** for pagination.

### 5.1 Pagination Requirement

We have an endpoint:

```ts
@Get()
getUsers(
  @Query('limit') limit: string,
  @Query('page') page: string,
) { /* ... */ }
```

We want:

* `limit` → how many records per page (e.g., 10, 30)
* `page` → which page (1, 2, 3, ...)

Examples:

* `GET /users?limit=10&page=1` → first page, 10 users
* `GET /users?limit=10&page=2` → second page, next 10 users
* `GET /users?limit=30&page=3` → third page, 30 users

`limit` and `page` are read as **strings**, but we want **numbers**.

### 5.2 Using `ParseIntPipe` on Query Params

```ts
@Get()
getUsers(
  @Query('limit', ParseIntPipe) limit: number,
  @Query('page', ParseIntPipe) page: number,
) {
  const usersService = new UsersService();
  console.log(limit, page);
  return usersService.getAllUsers();
}
```

* `@Query('limit', ParseIntPipe)`:

  * Reads `limit` from the query string.
  * Converts it to an integer.
* `@Query('page', ParseIntPipe)`:

  * Reads `page` and converts to integer.
* Inside the method, both `limit` and `page` are **numbers**.

If the user passes a non-numeric value (e.g., `limit=abc`):

* `ParseIntPipe` will throw a `400 Bad Request` error.

---

## 6. `DefaultValuePipe` – Setting Default Values

### 6.1 The Problem

What if the client **does not** send `limit` or `page`?

Example:

* Request: `GET /users` (no query params)
* Then:

  * `limit` will be `undefined`
  * `page` will be `undefined`
* If `ParseIntPipe` runs on `undefined`, it will throw an error because `undefined` cannot be converted to integer.

We want a behaviour like:

* If `limit` is **not provided**, use default value: `10`
* If `page` is **not provided**, use default value: `1`

### 6.2 Using `DefaultValuePipe`

`DefaultValuePipe` allows us to set **fallback values**.

```ts
@Get()
getUsers(
  @Query('limit', new DefaultValuePipe(10), ParseIntPipe) limit: number,
  @Query('page', new DefaultValuePipe(1), ParseIntPipe) page: number,
) {
  const usersService = new UsersService();
  console.log(limit, page);
  return usersService.getAllUsers();
}
```

### 6.3 Important Details

1. **Import and usage**:

   * `DefaultValuePipe` is imported from `@nestjs/common`.

   * It must be instantiated with `new`:

     ```ts
     new DefaultValuePipe(10)
     ```

   * This is different from other pipes like `ParseIntPipe`, which can be passed **directly** (without `new`).

2. **Order of pipes matters**:

   * This is critical:

     ```ts
     // ❌ Wrong order (causes error if value is missing)
     @Query('limit', ParseIntPipe, new DefaultValuePipe(10))

     // ✅ Correct order
     @Query('limit', new DefaultValuePipe(10), ParseIntPipe)
     ```

   * NestJS executes pipes **in the order they are listed**.

   * If `ParseIntPipe` runs **before** `DefaultValuePipe` and the value is missing:

     * The value is `undefined` → `ParseIntPipe` fails → `400 Bad Request`.

   * If `DefaultValuePipe` runs **first**:

     * It replaces `undefined` with the default (`10` or `1`).
     * Then `ParseIntPipe` converts that default value to `number`.

3. **Resulting behavior**:

   * Request: `GET /users` (no `limit`, no `page`)

     * `limit` → `DefaultValuePipe(10)` → `10` → `ParseIntPipe` → `10` (number)
     * `page` → `DefaultValuePipe(1)` → `1` → `ParseIntPipe` → `1` (number)
     * Controller receives: `limit = 10`, `page = 1`.

   * Request: `GET /users?limit=30&page=3`

     * Values provided → DefaultValuePipe is skipped because value is not `undefined`.
     * `ParseIntPipe` converts `'30'` → `30`, `'3'` → `3`.

---

## 7. Error Handling Example with `ParseIntPipe`

### 7.1 Invalid Route Parameter

* Request:

  ```txt
  GET /users/abc
  ```

* `@Param('id', ParseIntPipe)` will:

  * Try to convert `'abc'` to integer.
  * Fail → throw error.

* Response details:

  * **Status code:** `400`
  * **Error message:** `Bad Request`

### 7.2 Valid but Non-existing ID

* Request:

  ```txt
  GET /users/100
  ```

* `ParseIntPipe` successfully converts `'100'` to `100`.

* No pipe error.

* If user with ID `100` does not exist:

  * That is handled by your **business logic**, not by the pipe.

---

## 8. Summary – Key Takeaways

1. **Pipes run before controller methods** and can **transform** or **validate** request data.
2. NestJS provides **9 built-in pipes**. In this note, we focused on:

   * `ParseIntPipe`
   * `DefaultValuePipe`
3. `ParseIntPipe`:

   * Converts route/query parameters from **string → integer**.
   * On failure, automatically returns **400 Bad Request**.
4. `DefaultValuePipe`:

   * Sets a **default value** when a parameter or query is not provided.
   * Must be instantiated using `new DefaultValuePipe(value)`.
5. **Order of pipes is important**:

   * Always put `DefaultValuePipe` **before** `ParseIntPipe` when both are used together.
6. Only `DefaultValuePipe` and `ValidationPipe` require `new` when used, others like `ParseIntPipe` can be used directly.
7. Using these built-in pipes makes your code:

   * Cleaner (no manual conversions)
   * Safer (automatic validation and error handling)
   * Easier to maintain.

---